In [ ]:
from tensorflow.keras import models, layers
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from functools import partial
from tensorflow.keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [ ]:
def build_model2D_basic(train_X, train_y, val_X, val_Y,
                kernel, activation, padding, pooling, batch_normalization, drop_out, optimizer, learning_rate, batch_size, epochs=200, val_data=True): # hyperparameter
    
    DefaultConv2D = partial(layers.Conv2D, kernel_size=kernel, activation=activation, padding=padding)
    
    model = models.Sequential()
    
   
    model.add(DefaultConv2D(filters=64, input_shape=(X_train_gadf[0].shape[0], X_train_gadf[0].shape[1], X_train_gadf[0].shape[2])))
    model.add(DefaultConv2D(filters=128))
    if pooling=="max":
        model.add(layers.MaxPooling2D((2,2)))
    elif pooling=="average":
        model.add(layers.AveragePooling2D((2,2)))

    if batch_normalization==True:
        model.add(layers.BatchNormalization())

    model.add(layers.Flatten())

    # hier nochmal drop_out als "Hyperparameter"?

    model.add(layers.Dense(256, activation=activation))
    #layers.Dense(128, activation="relu"),

    if drop_out != None: 
        model.add(layers.Dropout(drop_out))

    # Output layer
    layers.Dense(3, activation="softmax")

    if optimizer=="adam":
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer =="RMSprop":
        optimizer=tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
    elif optimizer=="SGD":
        optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate)

    model.compile(optimizer=optimizer,
            loss="sparse_categorical_crossentropy",
            metrics=["accuracy"])
    
    # es = EarlyStopping(monitor='val_accuracy', mode='min', verbose=1, patience=10)
    # mcp = ModelCheckpoint("best_model", monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='max')
    # es2 = EarlyStopping(monitor='accuracy', mode='min', verbose=1, patience=10)
  
    if val_data == True: 
        model.fit(train_X, train_y, 
                   validation_data=(val_X, val_y ),
                   epochs=epochs, 
                   batch_size=batch_size,
                   class_weight=weights,
                   # callbacks=[es]
                ) 
    else: # used for retraining 
        model.fit(train_X, train_y, 
                   epochs=epochs, 
                   batch_size=batch_size,
                   class_weight=weights,
                  # callbacks=[es2]
                ) 
   
    return model

In [ ]:
grid2={"kernel": [(3,3),(5,5)],
      "activation":["relu", "sigmoid", "softmax"],
      "padding": ["SAME", "VALID"],
      "pooling": ["max", "average"], 
      "batch_normalization": [True, False],
      "drop_out": [None, 0.25, 0.5],
      "epochs": [5, 10, 25, 50, 75, 100, 150],
      "batch_size": [16, 32, 64],
      "optimizer": ["RMSprop",  "Adam", "SGD"], 
      "learning_rate": [0.0001, 0.001, 0.01] }

keys, values = zip(*grid2.items())
permut_grid = [dict(zip(keys, v)) for v in itertools.product(*values)]

np.random.RandomState(100)
random.shuffle(permut_grid)

In [ ]:
# new split for larger validation sets 
val_iterations=3
validation_size=len(X_train_gadf)*0.2
train_split_start = np.array([0, validation_size, validation_size*2]).astype("int")
train_split_end = np.array([-3*validation_size, -2*validation_size, -1*validation_size]).astype("int") # also val split_start 
val_split_end = np.array([-2*validation_size, -1*validation_size, -1]).astype("int")

In [ ]:
for cv in range(val_iterations):
    train_X = X_train_gadf[train_split_start[cv]:train_split_end[cv]]
    train_y = y_train[train_split_start[cv]:train_split_end[cv]]
    val_X = X_train_gadf[train_split_end[cv]:val_split_end[cv]]
    val_y = y_train[train_split_end[cv]:val_split_end[cv]]
    print(train_X.shape)
    print(train_y.shape)
    print(val_X.shape)
    print(val_y.shape)
    print()